In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report, accuracy_score, confusion_matrix

import torch
from unsloth import FastLanguageModel
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline

from unsloth import is_bfloat16_supported
from peft import LoraConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
system_prompt = (
    "You are an expert in financial sentiment analysis. Financial sentiment is defined as the tone and outlook expressed in financial texts. Sentiments can have a significant impact on market perceptions and investor decisions. Classify the following input text as positive, neutral, or negative based on its potential impact on financial markets and investor sentiment."
)



In [4]:
def prepare_prompt(row, train):
    prompt = system_prompt + "\n\n ### Input: " + row["Sentence"] + "\n\n ### Response: "
    if train:
         prompt = prompt + row["Sentiment"] # Add label
    return prompt

In [5]:
train_df = pd.read_csv("/content/train_data.csv")
val_df = pd.read_csv("/content/validation_data.csv")
test_df = pd.read_csv("/content/test_data.csv")

train_df["text"] = train_df.apply(lambda row: prepare_prompt(row, True), axis=1)
val_df["text"] = val_df.apply(lambda row: prepare_prompt(row, True), axis=1)
test_df["text"] = test_df.apply(lambda row: prepare_prompt(row, False), axis=1)


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

print(train_df["text"][101])
print(val_df["text"][101])
print(test_df["text"][101])

You are an expert in financial sentiment analysis. Financial sentiment is defined as the tone and outlook expressed in financial texts. Sentiments can have a significant impact on market perceptions and investor decisions. Classify the following input text as positive, neutral, or negative based on its potential impact on financial markets and investor sentiment.

 ### Input: Ramirent 's net sales in the second quarterended June 30 were EURO 128.7 million about U.S. $ 163 million , a 3.3-percent increase compared with EURO 124.6 million for thesecond quarter last year .

 ### Response: positive
You are an expert in financial sentiment analysis. Financial sentiment is defined as the tone and outlook expressed in financial texts. Sentiments can have a significant impact on market perceptions and investor decisions. Classify the following input text as positive, neutral, or negative based on its potential impact on financial markets and investor sentiment.

 ### Input: Vanhanen said the s

In [6]:
def evaluate(y_true, y_pred):
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none':1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')


    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [40]:
def predict(model, tokenizer):
    y_pred = []
    FastLanguageModel.for_inference(model)
    for i in tqdm(range(len(test_df))):
        prompt = test_df["text"][i]
        inputs = tokenizer(
            [
              prompt
            ], return_tensors = "pt").to("cuda")

        outputs = model.generate(**inputs, max_new_tokens = 1, use_cache = True)
        result = tokenizer.batch_decode(outputs)
        answer = result[0].split("### Response: ")[1].strip()
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred

In [11]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
y_pred = predict(model, tokenizer)

100%|██████████| 877/877 [02:22<00:00,  6.14it/s]


In [10]:
y_true = test_df["Sentiment"]
evaluate(y_true, y_pred)

Accuracy: 0.319

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       129
           1       1.00      0.00      0.01       470
           2       0.32      1.00      0.48       278

    accuracy                           0.32       877
   macro avg       0.44      0.33      0.16       877
weighted avg       0.64      0.32      0.16       877


Confusion Matrix:
[[  0   0 129]
 [  0   2 468]
 [  0   0 278]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [13]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [14]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params = {trainable_params} | All params = {all_param} | Trainable % = {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)

Trainable params = 41943040 | All params = 4582543360 | Trainable % = 0.92


In [15]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        seed = 3407,
        output_dir = "./output",
        logging_dir="./logs"
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/4089 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/876 [00:00<?, ? examples/s]

In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,089 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,022
 "-____-"     Number of trainable parameters = 41,943,040


Epoch,Training Loss,Validation Loss
0,0.772200,0.747705
1,0.587000,0.734300


In [17]:
trainer.state.log_history

[{'loss': 2.8004,
  'grad_norm': 0.8742996454238892,
  'learning_rate': 1e-05,
  'epoch': 0.0019559902200488996,
  'step': 1},
 {'loss': 2.8704,
  'grad_norm': 0.8632084131240845,
  'learning_rate': 2e-05,
  'epoch': 0.003911980440097799,
  'step': 2},
 {'loss': 2.7849,
  'grad_norm': 0.8282110095024109,
  'learning_rate': 3e-05,
  'epoch': 0.0058679706601467,
  'step': 3},
 {'loss': 3.0619,
  'grad_norm': 0.9056271314620972,
  'learning_rate': 4e-05,
  'epoch': 0.007823960880195598,
  'step': 4},
 {'loss': 2.8672,
  'grad_norm': 0.9479199647903442,
  'learning_rate': 5e-05,
  'epoch': 0.009779951100244499,
  'step': 5},
 {'loss': 2.7601,
  'grad_norm': 0.9151685237884521,
  'learning_rate': 4.9950835791543757e-05,
  'epoch': 0.0117359413202934,
  'step': 6},
 {'loss': 2.9974,
  'grad_norm': 1.110748052597046,
  'learning_rate': 4.990167158308752e-05,
  'epoch': 0.013691931540342298,
  'step': 7},
 {'loss': 2.8151,
  'grad_norm': 1.1860450506210327,
  'learning_rate': 4.985250737463127

In [18]:
output_dir = "./trained_model"
trainer.save_model()
tokenizer.save_pretrained(output_dir)

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/tokenizer.json')

In [31]:
print(test_df["text"][10])
print(test_df["Sentiment"][10])

You are an expert in financial sentiment analysis. Financial sentiment is defined as the tone and outlook expressed in financial texts. Sentiments can have a significant impact on market perceptions and investor decisions. Classify the following input text as positive, neutral, or negative based on its potential impact on financial markets and investor sentiment.

 ### Input: The board of directors also proposed that a dividend of EUR0 .20 per outstanding share be paid .

 ### Response: 
neutral


In [44]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    test_df["text"][10]
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1, use_cache = True)
result = tokenizer.batch_decode(outputs)
result

['<|begin_of_text|>You are an expert in financial sentiment analysis. Financial sentiment is defined as the tone and outlook expressed in financial texts. Sentiments can have a significant impact on market perceptions and investor decisions. Classify the following input text as positive, neutral, or negative based on its potential impact on financial markets and investor sentiment.\n\n ### Input: The board of directors also proposed that a dividend of EUR0.20 per outstanding share be paid.\n\n ### Response:  neutral']

In [41]:
y_pred = predict(model, tokenizer)


100%|██████████| 877/877 [02:23<00:00,  6.12it/s]


In [43]:
evaluate(y_true, y_pred)

Accuracy: 0.806

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.28      0.43       129
           1       0.76      0.94      0.84       470
           2       0.88      0.83      0.86       278

    accuracy                           0.81       877
   macro avg       0.85      0.68      0.71       877
weighted avg       0.82      0.81      0.79       877


Confusion Matrix:
[[ 36  88   5]
 [  4 441  25]
 [  0  48 230]]


Accuracy improved from 31.9% to 80.6%